In [1]:
from elasticsearch import Elasticsearch
import nltk
import math
import sys
import enchant
d = enchant.Dict("en_US")

In [3]:
es = Elasticsearch(['http://localhost:9200/'])
doc = {
        'size' : 10000,
        'query': {
            'match_all' : {}
       }
   }
res = es.search(index='newsgroup', doc_type='document', body=doc,scroll='1m')

In [4]:
def accept_sentence(sentence_divergent_dict):
    max_divergence = 0
    ret_sentence = ""
    for sentence in sentence_divergent_dict:
        if sentence_divergent_dict[sentence] > max_divergence:
            max_divergence = sentence_divergent_dict[sentence]
            ret_sentence = sentence
    return ret_sentence

In [5]:
summaries = {}
for item in res['hits']['hits']:
    text = item['_source']['doc_text']
    id_ = item['_source']['doc_id']
    sentences = text.split(".")
    text = nltk.tokenize.word_tokenize(text)
    for word in text:
        if not d.check(word):
            if len(word) < 4:
                text.remove(word)
    global_dist = nltk.FreqDist(text)
    i = 0
    summary = ""
    while i < 10 and len(sentences) != 0:
        sentence_divergence_dict = {}
        for sentence in sentences:
            merged_text = nltk.tokenize.word_tokenize(summary + sentence)
            merged_dist = nltk.FreqDist(merged_text)
            divergence = 0
            for word in global_dist:
                if word in merged_dist:
                    if merged_dist[word] != 0:
                        divergence += global_dist[word] * math.log(global_dist[word] / merged_dist[word])
            sentence_divergence_dict[sentence] = divergence
        sentence_to_add = accept_sentence(sentence_divergence_dict)
        summary += sentence_to_add + ". "
        if sentence_to_add in sentences:
            sentences.remove(sentence_to_add)
        i += 1
    summaries[id_] = summary

In [11]:
summaries['3']

' So why search a plane for weapons\t        since it\'s content is announced to be weapons?   \tIf there is one that\'s confused then that\'s you! We have the right (and we do)\tto give weapons to the Azeris, since Armenians started the fight in Azerbadjan! |>You are correct, all Turkish planes should be simply shot down! Nice, slow|>moving air transports!\tShoot down with what? Armenian bread and butter? Or the arms and personel \tof the Russian army?. \tThe area will be "greater" after some years, like your "holocaust" numbers. \ti        Turkey\'s government has announced that it\'s giving weapons  <-----------i        to Azerbadjan since Armenia started to attack Azerbadjan\t\t        it self, not the Karabag province.  \t\t\t    ***************\tNOTHING OF THE MENTIONED IS TRUE, BUT LET SAY IT\'s TRUE. . . . . . . '

In [12]:
docs_in_elastic = {}
for item in res['hits']['hits']:
    id_ = item['_source']['doc_id']
    docs_in_elastic[id_] = item['_source']['doc_text']

In [15]:
es.indices.delete(index='newsgroup', ignore=[400, 404])
mappings_duc = {
    'mappings':{
        'document':{
            'properties':{
                'doc_id': {'type': 'text', 'index': 'false'},
                'doc_text': {'type': 'text', 'analyzer': 'english'},
                'kl_summary':{'type': 'text', 'analyzer': 'english'}
            }
        }
    }
}
es.indices.create(index="newsgroup", body=mappings_duc)

{'acknowledged': True, 'index': 'newsgroup', 'shards_acknowledged': True}

In [16]:
docs = []
for id_ in docs_in_elastic:
    docs.append({
        'doc_id' : id_,
        'doc_text': docs_in_elastic[id_],
        'kl_summary' : summaries[id_]
    })
for doc in docs:
    es.index(index='newsgroup', doc_type='document', body=doc)